In [ ]:
import optuna
import vectorbt as vbt
from datetime import datetime, timedelta


from generator import BacktestDataGenerator

from strategy.kama import KamaStrategy
from optimisation.base import OptimizerSimple
from optimisation.walk_forward import WalkForwardOptimizer

from sampling import Sampling
from utils import OptimConfig


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
SIZE = 0
INTERVAL = '5m'
START = datetime(2025, 1, 1)
END = datetime(2025, 1, 2)
generator = BacktestDataGenerator(interval=INTERVAL, size=SIZE,
                                  start_date=START, end_date=END)
data = generator.get_data("BTCUSDT")



strategy = KamaStrategy(data)
pf = strategy.run(20, 50)

2025-04-26 14:11:11,250 - utils - INFO - DataGenerator initialized ✅
2025-04-26 14:11:31,513 - utils - INFO - Data loaded from DB for BTCUSDT: 288 bars ✅


In [38]:
pf.positions.plot()

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '0186db07-0c0b-424e-9722-a20b696d9578',
              'x': array([datetime.datetime(2025, 1, 1, 4, 15),
                          datetime.datetime(2025, 1, 1, 4, 20),
                          datetime.datetime(2025, 1, 1, 4, 25), ...,
                          datetime.datetime(2025, 1, 1, 23, 50),
                          datetime.datetime(2025, 1, 1, 23, 55),
                          datetime.datetime(2025, 1, 2, 0, 0)], shape=(238,), dtype=object),
              'y': array([93681.8, 93670. , 93710.3, ..., 94575. , 94580.9, 94457.7], shape=(238,))},
             {'customdata': array([[0, 0.0010674431960103242, 0.0, 'Long'],
                                   [1, 0.0010645565138004997, 0.0, 'Long'],
                                   [2, 0.0010603394141186092, 0.0, 'Long'],
                                

In [40]:
#pf.orders.records_readable

strategy.viz(20, 50)

In [7]:
# Résultats
#print(pf.stats())
pf.plot().show()

In [ ]:
strategy = KamaStrategy

optimizer = OptimizerSimple(strategy, data)
best_params = optimizer.optimize(n_trials=20)

print("Best params:", best_params)


In [ ]:
optim_config = OptimConfig(optimization_period=6,
                           gap_period=5,
                           validation_period=2,
                           n_splits=2)

walk_forward_optimizer = WalkForwardOptimizer(start=START, end=END,
                                              optim_config=optim_config,
                                              strategy_class=strategy,
                                              data=data)
results = walk_forward_optimizer.walk_forward(n_trials=4)
print(results)